# MCMC Estimation of Moments of $W$

In this notebook, we run the simulations to compute an MCMC estimate of the moments of $W$. In addition to the files contained in this repository, you will need `hypergraph.py` (available [here](https://github.com/PhilChodrow/hypergraph)) in the `scripts` directory, and an unzipped copy of the `contact-high-school` data set (available [here](https://www.cs.cornell.edu/~arb/data/contact-high-school/index.html)) in the `data` directory. 

Please note that running this notebook as written may take a long time -- about 3 days on the author's Celeron 2955U processor. Use of a remote computing runtime is recommended.  

In [2]:
import numpy as np
from scripts import hypergraph, read
from scripts.utils import experiment
import pathlib

In [3]:
data = 'contact-high-school'
through_path = 'throughput/' + data

In [4]:
# create the throughput folder for 
pathlib.Path(through_path).mkdir(parents=True, exist_ok=True) 

In [5]:
# subset for convenience purposes -- data subset will start at this timestamp
t_min = 1386330122

# read in the data, represent it as a hypergraph object, and compute the projected dyadic graph (calling it G)
C = read.read_data(data, prefix = 'data/', t_min = t_min)
C = hypergraph.hypergraph(C)
G = hypergraph.projected_graph(C, as_hyper = True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/contact-high-school/data/contact-high-school-nverts.txt'

In [ ]:
# conduct the simulation

n_steps = round(G.m*np.log(G.m)) # heuristic from coupon-collector problems. 
w, x, w2 = experiment(G, 
                      n_stub = 10**6,          # do stub-labeled MCMC as a warm-start 
                      n_vertex = n_steps/10,   # steps per round
                      n_rounds = 10**5,        # number of sampling rounds
                      sample_after = 50,       # give some time to mix
                      message_every = 10**4)   # print a periodic message

In [ ]:
# save the results

np.savetxt(through_path + '/w.txt',  w,  "%.4f")
np.savetxt(through_path + '/x.txt',  x,  "%.4f")
np.savetxt(through_path + '/w2.txt', w2, "%.4f")